# Sequential single-point Bayesian optimisation

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import lbfgsb
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # optimise acquisition function
    x_new, _ = lbfgsb(func=acq, bounds=bounds, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 34: 	 Inputs: [0.1944 0.2627 0.     0.311  0.3311 0.7252], 	 Outputs: [-2.2019]
New best at evaluation 35: 	 Inputs: [0.2237 0.1796 0.     0.3295 0.3233 0.6819], 	 Outputs: [-2.3608]
New best at evaluation 36: 	 Inputs: [0.2334 0.1673 0.501  0.3297 0.3209 0.6827], 	 Outputs: [-3.194]
New best at evaluation 40: 	 Inputs: [0.2061 0.1872 0.513  0.317  0.2864 0.642 ], 	 Outputs: [-3.2033]
New best at evaluation 41: 	 Inputs: [0.2555 0.1491 0.4892 0.2897 0.2878 0.6616], 	 Outputs: [-3.2528]
New best at evaluation 42: 	 Inputs: [0.1926 0.161  0.5281 0.2637 0.3057 0.683 ], 	 Outputs: [-3.2734]
New best at evaluation 47: 	 Inputs: [0.2236 0.1315 0.4595 0.2746 0.3192 0.6622], 	 Outputs: [-3.3069]
New best at evaluation 48: 	 Inputs: [0.1971 0.1513 0.4775 0.278  0.3096 0.6462], 	 Outputs: [-3.3184]
New best at evaluation 53: 	 Inputs: [0.207  0.1481 0.4719 0.2767 0.3093 0.6667], 	 Outputs: [-3.3193]
New best at evaluation 55: 	 Inputs: [0.2069 0.1546 0.4753 0.2798 0.3106 0